# Speech Emotion Recognizer

## Importing necessary packages

In [7]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## Defining a function to extract vocal features
This function will use mel spectrograms to extract the defining features of each emotion. Data can be found [here](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [8]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result,mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

## Creating a dictionary of emotions
We isolated 4 specific emotions to be our observed emotions

In [9]:
emotions = {
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful','disgust']

## Defining a load_data function
This function loads in all of the .wav files and targets only our desire emotions. The feature and emotions as appended to lists and returns train_test_split of the data. 

In [10]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("./data/DataFlair/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [11]:
# Running the load_data function, with 25% being separated as the test set. 
x_train, x_test, y_train, y_test, = load_data(test_size=0.25)

In [12]:
# Checking the shape of the train and test sets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [18]:
# Seeing how many features were extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 168


## Intializing the model

In [14]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, 
                    hidden_layer_sizes=(300,),
                    learning_rate='adaptive',
                    max_iter=500)

In [15]:
# fitting the model on x and y train
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [16]:
# Using x test for prediction
y_pred = model.predict(x_test)

## Check the accuracy score

In [17]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 75.52%
